In [8]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#read csv file
data = pd.read_csv('Seattle_Real_Time_Fire_911_Calls.csv')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#print first and last rows of the data
print(data.head())

In [ ]:
# Get a summary of the dataset
print(data.info())

In [ ]:
# Check basic statistics of the numerical variables
print(data.describe())

In [ ]:
# Check for missing values
print(data.isnull().sum())

In [ ]:
sns.histplot(data['Datetime'], kde=True)
plt.show()

In [ ]:
sns.histplot(data['Latitude'], kde=True)
plt.show()

In [ ]:
sns.histplot(data['Longitude'], kde=True)
plt.show()

In [ ]:
sns.boxplot(x='Latitude', data=data)
plt.show()

In [ ]:
sns.boxplot(x='Longitude', data=data)
plt.show()

In [ ]:
plt.scatter(data['Latitude'], data['Longitude'])
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

In [ ]:
# convert to datetime
data["Datetime"] = pd.to_datetime(data["Datetime"])
data["DayOfWeek"] = data["Datetime"].dt.weekday
data["IsWeekend"] = data["DayOfWeek"] >= 5

# Count incidents by weekday/weekend
incident_counts = data.groupby('IsWeekend')['Incident Number'].count()

# Create a bar plot
sns.barplot(x=incident_counts.index, y=incident_counts.values)
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.xlabel('Day Type')
plt.ylabel('Incident Count')
plt.title('Incidents on Weekdays vs. Weekends')
plt.show()

In [ ]:
#top regions
top_regions_total = data['Address'].value_counts().head(5)
#top jurisdictions
top_regions_jurisdiction = data.groupby('Address')['Incident Number'].count().nlargest(5)
#incidents per year
data['Year'] = pd.to_datetime(data['Datetime']).dt.year
top_regions_yearly = data.groupby(['Year', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per month
data['Month'] = pd.to_datetime(data['Datetime']).dt.month
top_regions_monthly = data.groupby(['Month', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per week
#data['Week'] = pd.to_datetime(data['Datetime']).dt.week
#top_regions_weekly = df.groupby(['Week', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)
#incidents per day
data['Day'] = pd.to_datetime(data['Datetime']).dt.date
top_regions_daily = data.groupby(['Day', 'Address'])['Incident Number'].count().groupby(level=0, group_keys=False).nlargest(5)

In [ ]:
print(top_regions_daily .head())

In [ ]:
print(top_regions_yearly .head())

In [ ]:
print(top_regions_monthly.head())

In [ ]:
#popular incidents on a daily basis
daily_popular_incidents = data.groupby(data['Datetime'].dt.date)['Type'].value_counts().groupby(level=0).idxmax()
print(daily_popular_incidents.head())

In [ ]:
#popular incidents on weekdays and weekends
weekday_popular_incidents = data[data['IsWeekend'] == False]['Type'].value_counts().idxmax()
weekend_popular_incidents = data[data['IsWeekend'] == True]['Type'].value_counts().idxmax()
print(weekday_popular_incidents)
print(weekend_popular_incidents)

In [ ]:
# regions that qualify for mobile emergency patrols
data['Week'] = data['Datetime'].dt.strftime('%Y-%U')
incident_counts = data.groupby(['Address', 'Type', 'Week']).size().reset_index(name='Count')
qualified_regions = incident_counts[incident_counts['Count'] > 50]['Address'].unique()
print(qualified_regions.tolist())

In [ ]:
#how much revenue the private company makes for every year of its operation
medevac_counts = (data['Type'] == 'Medic Response').groupby(data['Year']).size()
firevac_counts = (data['Type'] == 'Fire Response').groupby(data['Year']).size()
evacuation_counts = medevac_counts + firevac_counts
rate_per_evacuation = 100
revenue_per_year = evacuation_counts * rate_per_evacuation
print(revenue_per_year)